<a href="https://colab.research.google.com/github/anshupandey/natural_language_processing/blob/master/Transfer_Learning_NLP_Universal_Sentence_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

In [0]:
url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(url)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
doc1 = ["India"]
doc2 = ["I love coding"]
doc3 = ["I love python porgamming and Data Science. Machine Learning is amazing."]
out1 = model(doc2)
out1.shape

TensorShape([Dimension(None), Dimension(512)])

In [0]:
out1

<tf.Tensor 'StatefulPartitionedCall_5:0' shape=(?, 512) dtype=float32>

In [0]:
with tf.Session() as sess:
  sess.run([tf.global_variables_initializer(),tf.tables_initializer()])
  output = sess.run(out1)
  print(output.shape)
  print(output[0][:10])

(1, 512)
[ 0.05635723 -0.04979894  0.08434804 -0.01508556  0.05099313 -0.02394584
 -0.09826893 -0.02082875  0.0238463   0.01335999]


In [0]:
# Text Classification

In [0]:
train_data = open("train_5500.txt").readlines()
test_data = open("test_data.txt").readlines()

In [0]:
len(train_data)

5452

In [0]:
len(test_data)

500

In [0]:
train_data[:5]

['DESC:manner How did serfdom develop in and then leave Russia ?\n',
 'ENTY:cremat What films featured the character Popeye Doyle ?\n',
 "DESC:manner How can I find a list of celebrities ' real names ?\n",
 'ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?\n',
 'ABBR:exp What is the full form of .com ?\n']

In [0]:
def prepare_data(data):
  x = []
  y = []
  for doc in data:
    docs = doc.split(" ")
    y.append(docs[0].split(":")[0])
    x.append(" ".join(docs[1:]).strip())
  return x,y


In [0]:
xtrain,ytrain = prepare_data(train_data)
xtest,ytest = prepare_data(test_data)

In [0]:
import pandas as pd

In [0]:
ytrain = pd.DataFrame(ytrain)
ytrain = pd.get_dummies(ytrain)
ytrain.head()

,0_ABBR,0_DESC,0_ENTY,0_HUM,0_LOC,0_NUM
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,0,0
4,1,0,0,0,0,0


In [0]:
ytest = pd.DataFrame(ytest)
ytest = pd.get_dummies(ytest)
ytest.head()

,0_ABBR,0_DESC,0_ENTY,0_HUM,0_LOC,0_NUM
0,0,0,0,0,0,1
1,0,0,0,0,1,0
2,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,0,0,0,0,1


In [0]:
categories = list(ytrain.columns)
categories = [i[2:] for i in categories]
categories

['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']

### Text Classification using Transfer learning

In [0]:
url = "https://tfhub.dev/google/universal-sentence-encoder/1"
emb = hub.Module(url)

In [0]:
# create a function which will accept a text doc and return the corresponding vector
def sent2vec(doc):
  vector = emb(tf.squeeze(tf.cast(doc,tf.string)),as_dict=True)['default']
  return vector

In [0]:
from tensorflow.keras import models,layers

In [0]:
input_layer = layers.Input(shape=(1,),dtype='string')
embedding_layer = layers.Lambda(sent2vec,output_shape=(512,))(input_layer)
dense1 = layers.Dense(256,activation='relu')(embedding_layer)
output_layer = layers.Dense(6,activation='softmax')(dense1)

model = models.Model(inputs=[input_layer],outputs=output_layer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
import numpy as np
xtrain = np.array(xtrain).reshape(-1,1)
xtest = np.array(xtest).reshape(-1,1)
print(xtrain.shape)
print(xtest.shape)

(5452, 1)
(500, 1)


In [0]:
with tf.Session() as sess:
  sess.run([tf.global_variables_initializer(),tf.tables_initializer()])
  history = model.fit(xtrain,ytrain,epochs=10,
                      batch_size=50,validation_data=(xtest,ytest),
                      verbose=True)
  model.save_weights("model.h5")

Train on 5452 samples, validate on 500 samples
Epoch 1/10
5452/5452 [==============================] - 6s 1ms/sample - loss: 0.9817 - acc: 0.7074 - val_loss: 0.4733 - val_acc: 0.8700
Epoch 2/10
5452/5452 [==============================] - 2s 302us/sample - loss: 0.5195 - acc: 0.8146 - val_loss: 0.3669 - val_acc: 0.8780
Epoch 3/10
5452/5452 [==============================] - 2s 301us/sample - loss: 0.4533 - acc: 0.8340 - val_loss: 0.3393 - val_acc: 0.8840
Epoch 4/10
5452/5452 [==============================] - 2s 297us/sample - loss: 0.4287 - acc: 0.8423 - val_loss: 0.3077 - val_acc: 0.8880
Epoch 5/10
5452/5452 [==============================] - 2s 291us/sample - loss: 0.4054 - acc: 0.8553 - val_loss: 0.3186 - val_acc: 0.8860
Epoch 6/10
5452/5452 [==============================] - 2s 290us/sample - loss: 0.3956 - acc: 0.8580 - val_loss: 0.3043 - val_acc: 0.8880
Epoch 7/10
5452/5452 [==============================] - 2s 298us/sample - loss: 0.3800 - acc: 0.8593 - val_loss: 0.2973 - val_a

In [0]:
new_doc = ["Who is president of United States of America?","What is price for Harley Davidson?"]
new_doc = np.array(new_doc).reshape(-1,1)
with tf.Session() as sess:
  sess.run([tf.global_variables_initializer(),tf.tables_initializer()])
  model.load_weights("model.h5")
  output = model.predict(new_doc)

In [0]:
output2 = [np.argmax(i) for i in output]
output2 = [categories[i] for i in output2]
output2

['HUM', 'NUM']